In [10]:
# 전날 대비 비율 계산 함수
def calculate_ratio(today, yesterday):
    if yesterday == 0:
        return 0
    else:
        return today / yesterday

def setRatio(df, newName, whatCol):
    df[whatCol] = pd.to_numeric(df[whatCol], errors='coerce').fillna(0)
    for i in range(1, len(df)):
        df.loc[i, newName] = calculate_ratio(df.loc[i, whatCol], df.loc[i - 1, whatCol])

In [11]:
def calculateNextDay(df, whatCol):
    df[whatCol] = pd.to_numeric(df[whatCol], errors='coerce').fillna(0)
    strNext = '다음날' + whatCol
    df[strNext] = 0
    for i in range(len(df) - 1):
        today_close = df.loc[i, whatCol]
        next_day_close = df.loc[i+1, whatCol]
        df.loc[i, strNext] = 1 if next_day_close > today_close else 0

In [12]:
def calculateDifferent(df, whatCol1, whatCol2):
    df[whatCol1] = pd.to_numeric(df[whatCol1], errors='coerce').fillna(0)
    df[whatCol2] = pd.to_numeric(df[whatCol2], errors='coerce').fillna(0)
    strNext = whatCol1 + '와' + whatCol2 + '차이'
    df[strNext] = 0.0
    for i in range(len(df)):
        df.loc[i, strNext] = abs(df.loc[i, whatCol1] - df.loc[i, whatCol2])

In [14]:
columns_to_convert = ['거래량', '거래대금', '시가', '고가', '저가', '종가']
for col in columns_to_convert:
    strRatio = '전날대비' + col + '비율'
    setRatio(money_2023_df, strRatio, col)

In [15]:
columns_to_next = ['종가','거래량','거래대금']
for col in columns_to_next:
    calculateNextDay(money_2023_df, col)

In [16]:
calculateDifferent(money_2023_df, '시가','고가')
calculateDifferent(money_2023_df, '시가','저가')
calculateDifferent(money_2023_df, '종가','고가')
calculateDifferent(money_2023_df, '종가','저가')

In [13]:
pip install pykrx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.2/254.2 kB 15.3 MB/s eta 0:00:00


In [17]:
from pykrx import stock

In [18]:
# 시작일과 종료일 설정
start_date = "2023-01-01"
end_date = "2023-12-31"

# 코스피 지수 데이터를 가져와 데이터프레임으로 저장
kospi_index = stock.get_index_ohlcv_by_date(start_date, end_date, "1001")[['종가']].dropna()

In [19]:
kospi_index.index = money_2023_df.index
money_2023_df['KOSPI종가'] = kospi_index

In [20]:
def setMoneyRatio(df, newName, whatCol):
    df[whatCol] = pd.to_numeric(df[whatCol], errors='coerce').fillna(0)
    for i in range(1, len(df)):
        df.loc[i, newName] = calculate_ratio(df.loc[i, whatCol]- df.loc[i - 1, whatCol], df.loc[i - 1, whatCol])

In [21]:
setMoneyRatio(money_2023_df, 'KOSPI지수월간수익률', 'KOSPI종가')
setMoneyRatio(money_2023_df, '월간수익률', '종가')

In [23]:
# 공분산
covariance = money_2023_df['월간수익률'].cov(money_2023_df['KOSPI지수월간수익률'])

# 분산
variance = np.var(money_2023_df['KOSPI지수월간수익률'])

# 베타
beta = covariance / variance

print("베타:", beta)

베타: 0.7703685116604451
